In [7]:
import qwiic_icm20948
import time
import sys
import numpy as np
import ipywidgets as widgets
from threading import Thread
from ellipsoid_fit import ellipsoid_fit, ellipsoid_plot, data_regularize
paused = False
collect = True

In [8]:
def toggleData(b):
    global paused
    global pause
    print(b)
    paused = False if paused else True
    pause.description = 'Resume' if paused else 'Pause'


In [ ]:
IMU = qwiic_icm20948.QwiicIcm20948()

if IMU.connected == False:
    print("The Qwiic ICM20948 device isn't connected to the system. Please check your connection", \
        file=sys.stderr)
Ascale = 65534/4.0
Gscale = 65534/500.0
Mscale = 10.0
accelData = []
gyroData = []
magData = []


In [ ]:
class dataCollector(Thread):
        
    def run(self):
        global collect
        global IMU
        global paused
        global accelData
        global gyroData
        global magData
        IMU.begin()
        collect = True
        while collect:
            if IMU.dataReady():
                IMU.getAgmt() # read all axis and temp from sensor, note this also updates all instance variables
                if not paused:   
                #    print(\
                # 'Accel X:{: 06f}g'.format(IMU.axRaw/Ascale)\
                #, '\n', 'Accel Y:{: 06f}g'.format(IMU.ayRaw/Ascale)\
                #, '\n', 'Accel Z:{: 06f}g'.format(IMU.azRaw/Ascale)\
                #, '\n', 'Gyro X{: 06f}dps'.format(IMU.gxRaw/Gscale)\
                #, '\n', 'Gyro Y{: 06f}dps'.format(IMU.gyRaw/Gscale)\
                #, '\n', 'Gyro Z{: 06f}dps'.format(IMU.gzRaw/Gscale)\
                #, '\n', 'Mag X{: 06f}'.format(IMU.mxRaw/Mscale)\
                #, '\n', 'Mag Y{: 06f}'.format(IMU.myRaw/Mscale)\
                #, '\n', 'Mag Z{: 06f}'.format(IMU.mzRaw/Mscale)\
                #)
                    accelData.append(np.array([IMU.axRaw/Ascale,IMU.ayRaw/Ascale,IMU.azRaw/Ascale]))
                    gyroData.append(np.array([IMU.gxRaw/Gscale,IMU.gyRaw/Gscale,IMU.gzRaw/Gscale]))
                    magData.append(np.array([IMU.mxRaw/Mscale,IMU.myRaw/Mscale,IMU.mzRaw/Mscale]))

                time.sleep(0.03)
            else:
                print("Waiting for data")
                time.sleep(0.5)


In [ ]:
collector = dataCollector()

In [10]:
pause = widgets.Button(description='Pause')
pause.on_click(toggleData)

display(pause)

Button(description='Pause', style=ButtonStyle())

Button(description='Pause', style=ButtonStyle())


In [ ]:
collector.start()

In [ ]:
#Stop Data Collection
collect = False

In [ ]:
npAccelData = np.array(accelData)
npGyroData = np.array(gyroData)
npMagData = np.array(magData)

In [ ]:
regAccelData = data_regularize(npAccelData)
#regGyroData = data_regularize(npGyroData)
regMagData = data_regularize(npMagData)


In [ ]:
accelCenter, accelTR,accelRadius = ellipsoid_fit(regAccelData)
magCenter, magTR,magRadius = ellipsoid_fit(regMagData)